OpenCV 사용


일단, 비디오를 불러와서 프레임단위로 추출 + 저장이 1차 목적

<참고>

https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html 




* videoCapture 인터페이스를 이용한 베이직 코드


opencv_source_code/samples/python2/video.py




* basic 비디오 처리 샘플코드


opencv_source_code/samples/python2/video_dmtx.py




* 멀티쓰레드 비디오 처리 샘플코드


opencv_source_code/samples/python2/video_threaded.py

대기시간
k = cv2.waitKey(0)
if k == 27: #esc key
cv2.destroyAllWindows()
break

In [1]:
import os

def createfolder(folder):
    try:
        if not os.path.exists(folder):
            os.makedirs(folder)
    except OSError:
        print('Error: creating directory' + folder)


In [2]:
import PIL
from PIL import Image

def load_image_file(file, mode='RGB'):
    im = PIL.Image.open(file)
    if mode:
        im = im.convert(mode)
    return np.array(im)

## path 경로 설정 해야해 미현아!!! 


import cv2

path = '/Users/hyeonjeong_baek/CyberProjects/Deep-t-ADIS/W8_ExtractFrame/sample3_frame'
video = cv2.VideoCapture('sample3.mp4')
count = 0

videoframe = video.get(cv2.CAP_PROP_POS_FRAMES)
videocount = video.get(cv2.CAP_PROP_FRAME_COUNT)

createfolder(path)

while video.isOpened():
    #ret 은 프레임이 존재하지 않을때 T/F 반환
    #frame 은 프레임 반환
    if(videoframe == videocount):
        video.open('sample3.mp4')
    
    ret, frame = video.read()
    if ret == False:
        print("Frame doesn't Exist")
        break
    
    try:
        cv2.imshow('sample3', frame)        
        cv2.imwrite('sample3_frame/frame %d.jpg' %count, frame)
        #print('Saved frame%d.jpg' %count)
        count += 1
        
    except :
        pass

video.release()
cv2.destroyAllWindows()



**face_recognition 의 load_image_file 함수가 안먹어서 api 코드보고 바꿈**

import os

directory = os.walk(path).__next__()[2]
print(len(directory))

## 샘플(연희얼굴 - frame 두 개만|)

import face_recognition

images = []
for i in range(2):
    image = load_image_file("sample3_frame/frame "+str(i)+".jpg")
    images.append(image)

face_locations = []
for img in images:
    print(img)
    face_location = face_recognition.face_locations(img)
    #face_locations = face_recognition.face_locations(image)
    face_locations.append(face_location)

    
print(face_locations)
print("I found {} face(s) in this photograph.".format(len(face_locations)))

for face_location in face_locations:
    print(face_location)
    #print("image", image)
    # Print the location of each face in this image
    top, right, bottom, left = face_location[0]
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

## face_recognition 라이브러리를 이용해서 face frame 추출

**지금은 video -> frame 이미지 저장 -> 이미지 불러와서 얼굴만 따오기** 

이거를 아예 video -> 얼굴 프레임 저장으로 간추릴 예정임

import numpy as np
import face_recognition


#image = face_recognition.load_image_file("/sample2_frame/frame 0.jpg")

images = []
#for i in range(len(os.walk(path).__next__()[2])):
for i in range(100):
    image = load_image_file("sample3_frame/frame "+str(i)+".jpg")
    images.append(image)
    

face_locations = []
for img in images:
    face_location = face_recognition.face_locations(img, model="cnn")
    
    '''여기 model="cnn" 하면 원래 된다는데 내 컴퓨터로는 안됨 ㅜㅜㅜㅜ'''
    
    
    #face_locations = face_recognition.face_locations(image)
    #print("I found {} face(s) in this photograph.".format(len(face_locations)))
    face_locations.append(face_location)
    
print(face_locations)

for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location[0]
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

https://face-recognition.readthedocs.io/en/latest/_modules/face_recognition/api.html#load_image_file 

## *여기만 하면 된다!!!


## mp4 파일 -> face frame 으로 바로 저장

In [ ]:
import cv2
import numpy as np
import face_recognition

path = '/Users/hyeonjeong_baek/CyberProjects/Deep-t-ADIS/W8_ExtractFrame/sample3_frame'
videoname = 'sample3'
video = cv2.VideoCapture(videoname+'.mp4')
count = 0
faces = []

videoframe = video.get(cv2.CAP_PROP_POS_FRAMES)
videocount = video.get(cv2.CAP_PROP_FRAME_COUNT)

createfolder(path)

while video.isOpened():
    #ret 은 프레임이 존재하지 않을때 T/F 반환
    #frame 은 프레임 반환
    if(videoframe == videocount):
        video.open(videoname+'.mp4')
    
    ret, frame = video.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    print(frame)
    if ret == False:
        print("Frame doesn't Exist")
        break
    
    try:
        cv2.imshow(videoname, frame)
        
        #frame -> 얼굴 detection 한 프레임으로 
        #face_location 은 얼굴 위치(행렬에서)
        face_location = face_recognition.face_locations(frame)
        top, right, bottom, left = face_location[0]
        print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

        #face_location 찾아서 파일로 저장
        face_image = Image.fromarray(frame[top:bottom, left:right])
        #Image.fromarray(face_image).show()
        face_image.save(videoname+'_frame/frame %d.jpg' %count)
        #print('Saved frame%d.jpg' %count)
        count += 1
        
    except :
        pass

print(faces)
video.release()
cv2.destroyAllWindows()



[[[174 137 122]
  [174 137 122]
  [174 137 122]
  ...
  [175 143 127]
  [175 143 127]
  [175 143 127]]

 [[174 137 122]
  [174 137 122]
  [174 137 122]
  ...
  [176 144 128]
  [175 143 127]
  [174 142 126]]

 [[174 137 122]
  [174 137 122]
  [174 137 122]
  ...
  [176 144 128]
  [175 143 127]
  [174 142 126]]

 ...

 [[114  67  46]
  [113  66  45]
  [108  62  44]
  ...
  [189 190 220]
  [171 172 202]
  [158 159 189]]

 [[115  69  43]
  [115  69  43]
  [111  66  45]
  ...
  [190 191 221]
  [175 176 206]
  [162 163 193]]

 [[113  67  41]
  [114  68  42]
  [109  64  43]
  ...
  [192 193 223]
  [178 179 209]
  [169 170 200]]]
A face is located at pixel location Top: 384, Left: 776, Bottom: 705, Right: 1097
[[[174 137 122]
  [174 137 122]
  [176 139 124]
  ...
  [175 143 127]
  [175 143 127]
  [175 143 127]]

 [[174 137 122]
  [174 137 122]
  [174 137 122]
  ...
  [176 144 128]
  [175 143 127]
  [175 143 127]]

 [[174 137 122]
  [174 137 122]
  [174 137 122]
  ...
  [176 144 128]
  [175 143

A face is located at pixel location Top: 333, Left: 718, Bottom: 718, Right: 1103
[[[177 140 125]
  [177 140 125]
  [178 141 126]
  ...
  [177 145 129]
  [176 144 128]
  [175 143 127]]

 [[177 140 125]
  [177 140 125]
  [178 141 126]
  ...
  [177 145 129]
  [176 144 128]
  [175 143 127]]

 [[177 140 125]
  [177 140 125]
  [178 141 126]
  ...
  [176 144 128]
  [175 143 127]
  [173 141 125]]

 ...

 [[ 62  21   5]
  [ 63  22   6]
  [ 63  22   6]
  ...
  [190 191 221]
  [190 191 221]
  [190 191 221]]

 [[ 70  29  13]
  [ 71  30  14]
  [ 71  30  14]
  ...
  [191 192 222]
  [191 192 222]
  [191 192 222]]

 [[ 80  39  23]
  [ 81  40  24]
  [ 81  40  24]
  ...
  [183 184 214]
  [183 184 214]
  [183 184 214]]]
A face is located at pixel location Top: 333, Left: 718, Bottom: 718, Right: 1103
[[[179 140 126]
  [179 140 126]
  [179 140 126]
  ...
  [177 145 129]
  [176 144 128]
  [175 143 127]]

 [[179 140 126]
  [179 140 126]
  [179 140 126]
  ...
  [177 145 129]
  [176 144 128]
  [175 143 127]]

A face is located at pixel location Top: 384, Left: 740, Bottom: 705, Right: 1061
[[[180 142 125]
  [180 142 125]
  [180 142 125]
  ...
  [167 135 119]
  [167 135 119]
  [167 135 119]]

 [[181 143 126]
  [180 142 125]
  [180 142 125]
  ...
  [167 135 119]
  [167 135 119]
  [167 135 119]]

 [[181 143 126]
  [180 142 125]
  [180 142 125]
  ...
  [167 135 119]
  [166 134 118]
  [166 134 118]]

 ...

 [[ 81  40  24]
  [ 82  41  25]
  [ 82  41  25]
  ...
  [165 169 198]
  [165 169 198]
  [165 169 198]]

 [[ 90  49  33]
  [ 90  49  33]
  [ 89  48  32]
  ...
  [175 179 208]
  [175 179 208]
  [175 179 208]]

 [[ 98  57  41]
  [ 99  58  42]
  [ 97  56  40]
  ...
  [185 189 218]
  [183 187 216]
  [183 187 216]]]
A face is located at pixel location Top: 384, Left: 740, Bottom: 705, Right: 1061
[[[180 144 126]
  [179 143 125]
  [179 143 125]
  ...
  [168 136 120]
  [167 135 119]
  [167 135 119]]

 [[179 143 125]
  [179 143 125]
  [179 143 125]
  ...
  [167 135 119]
  [167 135 119]
  [167 135 119]]

A face is located at pixel location Top: 384, Left: 740, Bottom: 705, Right: 1061
[[[180 144 126]
  [179 143 125]
  [179 143 125]
  ...
  [170 139 120]
  [172 141 122]
  [172 141 122]]

 [[180 144 126]
  [180 144 126]
  [180 144 126]
  ...
  [170 139 120]
  [170 139 120]
  [170 139 120]]

 [[180 144 126]
  [180 144 126]
  [180 144 126]
  ...
  [169 138 119]
  [170 139 120]
  [170 139 120]]

 ...

 [[ 74  33  13]
  [ 74  33  13]
  [ 74  33  13]
  ...
  [178 180 207]
  [176 178 205]
  [174 176 203]]

 [[ 76  35  15]
  [ 76  35  15]
  [ 76  35  15]
  ...
  [185 187 214]
  [185 187 214]
  [181 183 210]]

 [[ 77  36  16]
  [ 77  36  16]
  [ 77  36  16]
  ...
  [182 184 211]
  [180 182 209]
  [180 182 209]]]
A face is located at pixel location Top: 384, Left: 740, Bottom: 705, Right: 1061
[[[180 144 126]
  [180 144 126]
  [180 144 126]
  ...
  [169 138 119]
  [170 139 120]
  [170 139 120]]

 [[180 144 126]
  [180 144 126]
  [180 144 126]
  ...
  [169 138 119]
  [169 138 119]
  [169 138 119]]

A face is located at pixel location Top: 384, Left: 740, Bottom: 705, Right: 1061
[[[181 145 127]
  [180 144 126]
  [180 144 126]
  ...
  [173 142 123]
  [173 142 123]
  [173 142 123]]

 [[181 145 127]
  [181 145 127]
  [180 144 126]
  ...
  [172 141 122]
  [172 141 122]
  [172 141 122]]

 [[181 145 127]
  [181 145 127]
  [180 144 126]
  ...
  [170 139 120]
  [170 139 120]
  [170 139 120]]

 ...

 [[ 74  33  15]
  [ 71  30  12]
  [ 70  29  11]
  ...
  [183 184 214]
  [184 185 215]
  [184 185 215]]

 [[ 73  32  14]
  [ 73  32  14]
  [ 71  30  12]
  ...
  [178 179 209]
  [185 186 216]
  [187 188 218]]

 [[ 71  30  12]
  [ 73  32  14]
  [ 73  32  14]
  ...
  [175 176 206]
  [180 181 211]
  [182 183 213]]]
A face is located at pixel location Top: 384, Left: 740, Bottom: 705, Right: 1061
[[[181 145 127]
  [181 145 127]
  [181 145 127]
  ...
  [173 142 123]
  [173 142 123]
  [173 142 123]]

 [[181 145 127]
  [181 145 127]
  [181 145 127]
  ...
  [172 141 122]
  [172 141 122]
  [172 141 122]]

A face is located at pixel location Top: 348, Left: 705, Bottom: 669, Right: 1026
[[[179 143 125]
  [179 143 125]
  [179 143 125]
  ...
  [170 139 120]
  [169 138 119]
  [168 137 118]]

 [[179 143 125]
  [179 143 125]
  [179 143 125]
  ...
  [169 138 119]
  [167 136 117]
  [166 135 116]]

 [[179 143 125]
  [179 143 125]
  [180 144 126]
  ...
  [165 134 115]
  [162 131 112]
  [162 131 112]]

 ...

 [[ 68  27  11]
  [ 70  29  13]
  [ 71  30  14]
  ...
  [180 181 211]
  [177 178 208]
  [183 184 214]]

 [[ 68  27  11]
  [ 70  29  13]
  [ 71  30  14]
  ...
  [185 186 216]
  [184 185 215]
  [190 191 221]]

 [[ 68  27  11]
  [ 70  29  13]
  [ 71  30  14]
  ...
  [187 188 218]
  [187 188 218]
  [192 193 223]]]
A face is located at pixel location Top: 348, Left: 705, Bottom: 669, Right: 1026
[[[180 144 126]
  [180 144 126]
  [180 144 126]
  ...
  [169 138 119]
  [167 136 117]
  [166 135 116]]

 [[180 144 126]
  [180 144 126]
  [180 144 126]
  ...
  [167 136 117]
  [165 134 115]
  [163 132 113]]

A face is located at pixel location Top: 348, Left: 740, Bottom: 669, Right: 1061
[[[180 144 126]
  [181 145 127]
  [183 147 129]
  ...
  [176 145 126]
  [174 143 124]
  [173 142 123]]

 [[181 145 127]
  [181 145 127]
  [181 145 127]
  ...
  [174 143 124]
  [174 143 124]
  [173 142 123]]

 [[183 147 129]
  [181 145 127]
  [180 144 126]
  ...
  [170 139 120]
  [169 138 119]
  [168 137 118]]

 ...

 [[ 44  18   6]
  [ 44  18   6]
  [ 44  18   6]
  ...
  [192 196 225]
  [188 192 221]
  [180 184 213]]

 [[ 46  20   8]
  [ 46  20   8]
  [ 46  20   8]
  ...
  [192 196 225]
  [190 194 223]
  [187 191 220]]

 [[ 49  23  11]
  [ 49  23  11]
  [ 49  23  11]
  ...
  [190 194 223]
  [190 194 223]
  [189 193 222]]]
A face is located at pixel location Top: 348, Left: 740, Bottom: 669, Right: 1061
[[[183 147 129]
  [183 147 129]
  [183 147 129]
  ...
  [174 143 124]
  [174 143 124]
  [174 143 124]]

 [[183 147 129]
  [183 147 129]
  [183 147 129]
  ...
  [174 143 124]
  [173 142 123]
  [173 142 123]]

A face is located at pixel location Top: 348, Left: 740, Bottom: 669, Right: 1061
[[[183 147 129]
  [181 145 127]
  [181 145 127]
  ...
  [175 141 123]
  [176 142 124]
  [177 143 125]]

 [[183 147 129]
  [181 145 127]
  [181 145 127]
  ...
  [176 142 124]
  [176 142 124]
  [176 142 124]]

 [[183 147 129]
  [181 145 127]
  [181 145 127]
  ...
  [176 142 124]
  [176 142 124]
  [176 142 124]]

 ...

 [[ 46  20   8]
  [ 46  20   8]
  [ 46  20   8]
  ...
  [197 198 228]
  [197 198 228]
  [197 198 228]]

 [[ 48  22  10]
  [ 48  22  10]
  [ 48  22  10]
  ...
  [197 198 228]
  [197 198 228]
  [197 198 228]]

 [[ 49  23  11]
  [ 49  23  11]
  [ 49  23  11]
  ...
  [195 196 226]
  [195 196 226]
  [195 196 226]]]
A face is located at pixel location Top: 348, Left: 740, Bottom: 669, Right: 1061
[[[181 145 127]
  [180 144 126]
  [180 144 126]
  ...
  [175 141 123]
  [176 142 124]
  [177 143 125]]

 [[181 145 127]
  [180 144 126]
  [180 144 126]
  ...
  [176 142 124]
  [176 142 124]
  [176 142 124]]

conda install cmake

conda install -c menpo dlib

conda install face_recognition


conda install pandas, pytorch, torchvision 등등